In [ ]:
from main import polygonize
import numpy as np

def get_directions(vecs):
    '''
    Returns two or three vectors specifying the direction in which each molecule should be aligned
    in the cyclical TS, pointing towards the center of the polygon.
    '''
    assert len(vecs) in (2,3)
    if len(vecs) == 2:
        return np.array([[0,1,0],
                            [0,-1,0]])
    else:
        a = vecs[0,1,0] # first vec, end, x
        b = vecs[1,1,0] # second vec, end, x
        c = vecs[1,1,1] # second vec, end, y

        x = a/2
        y = (b**2 + c**2 - a*b)/(2*c)
        cc = np.array([x,y,0])
        # coordinates of the triangle circocenter

        v1 = cc - np.mean((vecs[0,1],vecs[2,1]), axis=0)
        v2 = cc - np.mean((vecs[1,1],vecs[0,1]), axis=0)
        v3 = cc - np.mean((vecs[2,1],vecs[1,1]), axis=0)
        # versors connecting center of side with circocenter

        return np.vstack((v1,v2,v3))

# get_directions(polygonize([1,1,1])[0])
# polygonize([1,1,1])[0]

from utils import cartesian_product
rotation_steps = 2
a = cartesian_product(*[range(-rotation_steps, rotation_steps+1) for _ in range(3)])*45/rotation_steps
a[0:10]

In [ ]:
x = np.array([[[0.,         0. ,        0.        ],
  [1.97332738, 0.   ,      0.        ]],

 [[2.83868969, 4.15092487, 0.        ],
  [1.97332738, 0.        , 0.        ]],

 [[2.83868969, 4.15092487, 0.        ],
  [0.        , 0.        , 0.        ]]])

[np.linalg.norm(i[0]-i[1]) for i in x]

## Dataclass

In [ ]:
from main import Options
options = Options()
dir(options)
{var:options.__getattribute__(var) for var in dir(options) if var[0:2] != '__'}

## Options with numbers

In [ ]:
keywords_list = ['STEPS=5']
line = 'NOOPT STEPS=7 BYPASS'
keywords = [l.split('=')[0] for l in line.split()]
keywords

## options with commas/equals

In [ ]:
d = {0:[[1,'a']], 1:[[9,'a']]}
def _set_custom_orbs(orb_string):
    '''
    orb_string looks like 'a=2.345,b=3.456,c=2.22'

    '''
    pairs = [(piece.split('=')[0], float(piece.split('=')[1])) for piece in orb_string.split(',')]

    for letter, dist in pairs:
        for index in range(len(self.objects)):
            for pairing in self.pairing_dict[index]:

    # for each pairing specified by the user, check each pairing recorded
    # in the pairing_dict on that molecule.

                if pairing[1] == letter:
                    for reactive_atom, reactive_index in zip(self.objects[index].reactive_atoms_classes, self.objects[index].reactive_indexes):
                        if reactive_index == pairing[0]:
                            reactive_atom.init(self.objects[index], reactive_index, update=True, orb_dim=dist/2)

                # If the letter matches, look for the correct reactive atom on that molecule. When we find the correct match,
                # set the new orbital center with imposed distance from the reactive atom. The imposed distance is half the 
                # user-specified one, as the final atomic distances will be given by two halves of this length.


# _set_custom_orbs('a=2.345,b=3.456')
# d = {0:[[1,'a']], 1:[[9,'a']]}

'CLASHES(a=3.5)'[8:-1]

## SUPRAFAC Keyword

In [ ]:
l = [0,2,3,1]

for n in l:
    keep = [i for i in l if n >= i]
    if len(keep) == 2:
        b = [i in keep for i in l]
        print(keep)
        print(n)
        print(b)

## Rodrigues Formula

In [ ]:
import numpy as np
from utils import norm, rot_mat_from_pointer
def rodrigues(pivot, angle):
    '''
    Pivot is a shape (3,) array
    Angle in degrees
    '''

    p = norm(pivot)
    a = angle/180*np.pi
    x = np.array([1,0,0])
    y = np.array([0,1,0])
    z = np.array([0,0,1])

    v0 = x*np.cos(a) + np.cross(p, x) * np.sin(a) + p*(np.dot(p, x))*(1-np.cos(a))
    v1 = y*np.cos(a) + np.cross(p, y) * np.sin(a) + p*(np.dot(p, y))*(1-np.cos(a))
    v2 = z*np.cos(a) + np.cross(p, z) * np.sin(a) + p*(np.dot(p, z))*(1-np.cos(a))

    v0 = v0[..., None]
    v1 = v1[..., None]
    v2 = v2[..., None]

    return np.hstack((v0,v1,v2))
pivot = np.array([1,0,0])

In [ ]:
%timeit -r 100 rodrigues(pivot, 180)

In [ ]:
%timeit -r 100 rot_mat_from_pointer(pivot, 180)

In [ ]:
print(rodrigues(pivot, 55))
print(rot_mat_from_pointer(pivot, 55))

## Align vectors

In [ ]:
from scipy.spatial.transform import Rotation as R
v1 = np.array((1,0,0))
v2 = np.array((0,1,0))
R.align_vectors((v1,v2),(v1,-v2))[0].as_matrix()

## RMSD threshold
Looking for the best RMSD treshold for optimized structures obtainment

In [ ]:
from prune import prune_conformers
import numpy as np
import matplotlib.pyplot as plt
import os
from cclib.io import ccread
os.chdir('Resources/RMSD_test')

fig = plt.figure(figsize=(8,5))
plt.ylabel(r'% structures kept')
plt.xlabel('RMSD')

for filename in os.listdir():
    if filename[-4:] == '.xyz':
        data = ccread(filename)
        x, y = [], []
        for RMSD in np.arange(0.5,3,0.05):
            mask = prune_conformers(data.atomcoords, data.atomnos, max_rmsd=RMSD)[1]
            kept = len([m for m in mask if m == True])/len(mask)
            # s = f'kept {len([m for m in mask if m == True])}/{len(mask)}'
            y.append(kept)
            x.append(RMSD)
        plt.plot(x, y, label=filename)
plt.legend()

## MOPAC Berny
### Full list of KEYWORDS
        & - TURN NEXT LINE INTO KEYWORDS
        + - ADD ANOTHER LINE OF KEYWORDS
        0SCF - READ IN DATA, THEN STOP
        1ELECTRON- PRINT FINAL ONE-ELECTRON MATRIX
        1SCF - DO ONE SCF AND THEN STOP
        AIDER - READ IN AB INITIO DERIVATIVES
        AIGIN - GEOMETRY MUST BE IN GAUSSIAN FORMAT
        AIGOUT - IN ARC FILE, INCLUDE AB-INITIO GEOMETRY
        ANALYT - USE ANALYTICAL DERIVATIVES OF ENERGY WRT GEOMETRY
        AM1 - USE THE AM1 HAMILTONIAN
        BAR=n.n - REDUCE BAR LENGTH BY A MAXIMUM OF n.n
        BIRADICAL- SYSTEM HAS TWO UNPAIRED ELECTRONS
        BONDS - PRINT FINAL BOND-ORDER MATRIX
        C.I. - A MULTI-ELECTRON CONFIGURATION INTERACTION SPECIFIED
        Keywords
        CHARGE=n - CHARGE ON SYSTEM = n (e.g. NH4 => CHARGE=1)
        COMPFG - PRINT HEAT OF FORMATION CALCULATED IN COMPFG
        CONNOLLY - USE CONNOLLY SURFACE
        DEBUG - DEBUG OPTION TURNED ON
        DENOUT - DENSITY MATRIX OUTPUT (CHANNEL 10)
        DENSITY - PRINT FINAL DENSITY MATRIX
        DEP - GENERATE FORTRAN CODE FOR PARAMETERS FOR NEW ELEMENTS
        DEPVAR=n - TRANSLATION VECTOR IS A MULTIPLE OF BOND-LENGTH
        DERIV - PRINT PART OF WORKING IN DERIV
        DFORCE - FORCE CALCULATION SPECIFIED, ALSO PRINT FORCE MATRIX.
        DFP - USE DAVIDON-FLETCHER-POWELL METHOD TO OPTIMIZE GEOMETRIES
        DIPOLE - FIT THE ESP TO THE CALCULATED DIPOLE
        DIPX - X COMPONENT OF DIPOLE TO BE FITTED
        DIPY - Y COMPONENT OF DIPOLE TO BE FITTED
        DIPZ - Z COMPONENT OF DIPOLE TO BE FITTED
        DMAX - MAXIMUM STEPSIZE IN EIGENVECTOR FOLLOWING
        DOUBLET - DOUBLET STATE REQUIRED
        DRC - DYNAMIC REACTION COORDINATE CALCULATION
        DUMP=n - WRITE RESTART FILES EVERY n SECONDS
        ECHO - DATA ARE ECHOED BACK BEFORE CALCULATION STARTS
        EF - USE EF ROUTINE FOR MINIMUM SEARCH
        EIGINV -
        EIGS - PRINT ALL EIGENVALUES IN ITER
        ENPART - PARTITION ENERGY INTO COMPONENTS
        ESP - ELECTROSTATIC POTENTIAL CALCULATION
        ESPRST - RESTART OF ELECTROSTATIC POTENTIAL
        ESR - CALCULATE RHF UNPAIRED SPIN DENSITY
        EXCITED - OPTIMIZE FIRST EXCITED SINGLET STATE
        EXTERNAL - READ PARAMETERS OFF DISK
        FILL=n - IN RHF OPEN AND CLOSED SHELL, FORCE M.O. n
        TO BE FILLED
        FLEPO - PRINT DETAILS OF GEOMETRY OPTIMIZATION
        FMAT - PRINT DETAILS OF WORKING IN FMAT
        FOCK - PRINT LAST FOCK MATRIX
        FORCE - FORCE CALCULATION SPECIFIED
        GEO-OK - OVERRIDE INTERATOMIC DISTANCE CHECK
        GNORM=n.n- EXIT WHEN GRADIENT NORM DROPS BELOW n.n
        GRADIENTS- PRINT ALL GRADIENTS
        GRAPH - GENERATE FILE FOR GRAPHICS
        HCORE - PRINT DETAILS OF WORKING IN HCORE
        HESS=N - OPTIONS FOR CALCULATING HESSIAN MATRICES IN EF
        H-PRIO - HEAT OF FORMATION TAKES PRIORITY IN DRC
        HYPERFINE- HYPERFINE COUPLING CONSTANTS TO BE CALCULATED
        IRC - INTRINSIC REACTION COORDINATE CALCULATION
        ISOTOPE - FORCE MATRIX WRITTEN TO DISK (CHANNEL 9 )
        ITER - PRINT DETAILS OF WORKING IN ITER
        ITRY=N - SET LIMIT OF NUMBER OF SCF ITERATIONS TO N.
        IUPD - MODE OF HESSIAN UPDATE IN EIGENVECTOR FOLLOWING
        K=(N,N) - BRILLOUIN ZONE STRUCTURE TO BE CALCULATED
        KINETIC - EXCESS KINETIC ENERGY ADDED TO DRC CALCULATION
        LINMIN - PRINT DETAILS OF LINE MINIMIZATION
        LARGE - PRINT EXPANDED OUTPUT
        LET - OVERRIDE CERTAIN SAFETY CHECKS
        LOCALIZE - PRINT LOCALIZED ORBITALS
        MAX - PRINTS MAXIMUM GRID SIZE (23*23)
        MECI - PRINT DETAILS OF MECI CALCULATION
        MICROS - USE SPECIFIC MICROSTATES IN THE C.I.
        MINDO/3 - USE THE MINDO/3 HAMILTONIAN
        MMOK - USE MOLECULAR MECHANICS CORRECTION TO CONH BONDS
        MODE=N - IN EF, FOLLOW HESSIAN MODE NO. N
        MOLDAT - PRINT DETAILS OF WORKING IN MOLDAT
        MS=N - IN MECI, MAGNETIC COMPONENT OF SPIN
        MULLIK - PRINT THE MULLIKEN POPULATION ANALYSIS
        NLLSQ - MINIMIZE GRADIENTS USING NLLSQ
        NOANCI - DO NOT USE ANALYTICAL C.I. DERIVATIVES
        NODIIS - DO NOT USE DIIS GEOMETRY OPTIMIZER
        NOINTER - DO NOT PRINT INTERATOMIC DISTANCES
        NOLOG - SUPPRESS LOG FILE TRAIL, WHERE POSSIBLE
        NOMM - DO NOT USE MOLECULAR MECHANICS CORRECTION TO CONH BONDS
        NONR -
        NOTHIEL - DO NOT USE THIEL’S FSTMIN TECHNIQUE
        NSURF=N - NUMBER OF SURFACES IN AN ESP CALCULATION
        NOXYZ - DO NOT PRINT CARTESIAN COORDINATES
        NSURF - NUMBER OF LAYERS USED IN ELECTROSTATIC POTENTIAL
        OLDENS - READ INITIAL DENSITY MATRIX OFF DISK
        OLDGEO - PREVIOUS GEOMETRY TO BE USED
        OPEN - OPEN-SHELL RHF CALCULATION REQUESTED
        ORIDE -
        PARASOK - IN AM1 CALCULATIONS SOME MNDO PARAMETERS ARE TO BE USED
        PI - RESOLVE DENSITY MATRIX INTO SIGMA AND PI BONDS
        PL - MONITOR CONVERGENCE OF DENSITY MATRIX IN ITER
        PM3 - USE THE MNDO-PM3 HAMILTONIAN
        POINT=N - NUMBER OF POINTS IN REACTION PATH
        POINT1=N - NUMBER OF POINTS IN FIRST DIRECTION IN GRID CALCULATION
        POINT2=N - NUMBER OF POINTS IN SECOND DIRECTION IN GRID CALCULATION
        POLAR - CALCULATE FIRST, SECOND AND THIRD ORDER POLARIZABILITIES
        POTWRT - IN ESP, WRITE OUT ELECTROSTATIC POTENTIAL TO UNIT 21
        POWSQ - PRINT DETAILS OF WORKING IN POWSQ
        PRECISE - CRITERIA TO BE INCREASED BY 100 TIMES
        PULAY - USE PULAY’S CONVERGER TO OBTAIN A SCF
        QUARTET - QUARTET STATE REQUIRED
        QUINTET - QUINTET STATE REQUIRED
        RECALC=N - IN EF, RECALCULATE HESSIAN EVERY N STEPS
        RESTART - CALCULATION RESTARTED
        ROOT=n - ROOT n TO BE OPTIMIZED IN A C.I. CALCULATION
        ROT=n - THE SYMMETRY NUMBER OF THE SYSTEM IS n.
        SADDLE - OPTIMIZE TRANSITION STATE
        SCALE - SCALING FACTOR FOR VAN DER WAALS DISTANCE IN ESP
        SCFCRT=n - DEFAULT SCF CRITERION REPLACED BY THE VALUE SUPPLIED
        SCINCR - INCREMENT BETWEEN LAYERS IN ESP
        SETUP - EXTRA KEYWORDS TO BE READ OF SETUP FILE
        SEXTET - SEXTET STATE REQUIRED
        SHIFT=n - A DAMPING FACTOR OF n DEFINED TO START SCF
        SIGMA - MINIMIZE GRADIENTS USING SIGMA
        SINGLET - SINGLET STATE REQUIRED
        SLOPE - MULTIPLIER USED TO SCALE MNDO CHARGES
        SPIN - PRINT FINAL UHF SPIN MATRIX
        STEP - STEP SIZE IN PATH
        Keywords
        STEP1=n - STEP SIZE n FOR FIRST COORDINATE IN GRID CALCULATION
        STEP2=n - STEP SIZE n FOR SECOND COORDINATE IN GRID CALCULATION
        STO-3G - DEORTHOGONALIZE ORBITALS IN STO-3G BASIS
        SYMAVG - AVERAGE SYMMETRY EQUIVALENT ESP CHARGES
        SYMMETRY - IMPOSE SYMMETRY CONDITIONS
        T=n - A TIME OF n SECONDS REQUESTED
        THERMO - PERFORM A THERMODYNAMICS CALCULATION
        TIMES - PRINT TIMES OF VARIOUS STAGES
        T-PRIO - TIME TAKES PRIORITY IN DRC
        TRANS - THE SYSTEM IS A TRANSITION STATE
        (USED IN THERMODYNAMICS CALCULATION)
        TRIPLET - TRIPLET STATE REQUIRED
        TS - USING EF ROUTINE FOR TS SEARCH
        UHF - UNRESTRICTED HARTREE-FOCK CALCULATION
        VECTORS - PRINT FINAL EIGENVECTORS
        VELOCITY - SUPPLY THE INITIAL VELOCITY VECTOR IN A DRC CALCULATION
        WILLIAMS - USE WILLIAMS SURFACE
        X-PRIO - GEOMETRY CHANGES TAKE PRIORITY IN DRC
        XYZ - DO ALL GEOMETRIC OPERATIONS IN CARTESIAN COORDINATES.

In [ ]:
from optimization_methods import mopac_opt
from tscode import write_xyz
import numpy as np
import os
from cclib.io import ccread
os.chdir('Resources/SN2')
data = ccread('test_berny.xyz')
constrained_indexes = np.array(((0,18),(6,16)))

In [ ]:
newcoords = mopac_opt(data.atomcoords[0], data.atomnos, method='PM7 TS')[0]
with open('test_berny_out.xyz', 'w') as f:
    write_xyz(newcoords, data.atomnos, f, title='test_berny_out.xyz')
os.system('obabel test_berny_out.xyz -O test_berny_out.sdf')
os.system('gview test_berny_out.sdf')

In [ ]:
from spyrmsd.rmsd import rmsd
rmsd(data.atomcoords[0], newcoords, data.atomnos, data.atomnos, center=True, minimize=True)

## ASE NEB

In [1]:
import time

class timer():
    def __init__(self, title=''):
        self.title = title
    def __enter__(self):
        self.t1 = time.time()
    def __exit__(self, type, value, traceback):
        self.t2 = time.time()
        if self.title != '':
            s = f'{self.title} - {round(self.t2-self.t1, 3)} s'
        else:
            s = f'{round(self.t2-self.t1, 3)} s' 
        print(s)

In [29]:
import os
from cclib.io import ccread
os.chdir(r'C:\Users\Nik\Desktop\Coding\TSCoDe\Resources\DA\ase')
data = ccread('structure_0_start.xyz')
start = data.atomcoords[0]
end = ccread('structure_0_end.xyz').atomcoords[0]

In [73]:
from ase.dyneb import DyNEB
from ase.neb import NEB
from ase.optimize import *
from ase import Atoms
from ase.calculators.mopac import MOPAC
from parameters import MOPAC_COMMAND
from ase.visualize import view
from optimization_methods import write_xyz

def dump(filename, images, atomnos):
    with open(filename, 'w') as f:
                for image in images:
                    coords = image.get_positions()
                    write_xyz(coords, atomnos, f)

def ase_neb(reagents, products, atomnos, n_images=6, fmax=0.05, label='temp'):

    # Read initial and final states:
    initial = Atoms(atomnos, positions=reagents)
    final = Atoms(atomnos, positions=products)

    # Make a band consisting of n images:
    images = [initial]
    images += [initial.copy() for i in range(n_images)]
    images += [final]

    climbing_neb = DyNEB(images, fmax=fmax, climb=True,  method='eb', scale_fmax=1, allow_shared_calculator=True)

    # Interpolate linearly the positions of the middle images:
    climbing_neb.interpolate()
    # dump('interpolated.xyz', images, atomnos)
    
    # Set calculators for all images
    for i, image in enumerate(images):
        image.calc = MOPAC(label=label, command=f'{MOPAC_COMMAND} {label}.mop', method='PM7 GEO-OK')

    # Optimize:
    climbing_optimizer = LBFGS(climbing_neb, trajectory='ASE_NEB_test.traj')
    try:
        climbing_optimizer.run(fmax=fmax, steps=500)

    except Exception as e:
        print(e)

    dump('debug.xyz', images, atomnos)
    energies = [image.get_total_energy() for image in images]
    ts_id = energies.index(max(energies))
    # print(f'TS structure is number {ts_id}, energy is {max(energies)}')

    return images[ts_id].get_positions()

In [31]:
ts_coords = ase_neb(start, end, data.atomnos)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 10:14:04    -2271.401800*       6.1310
FIRE:    1 10:14:04    -2272.394960*       3.5221
FIRE:    2 10:14:05    -2272.847100*       4.5209
FIRE:    3 10:14:06    -2273.085800*       4.2573
FIRE:    4 10:14:06    -2273.339720*       2.3239
FIRE:    5 10:14:07    -2273.473440*       2.8806
FIRE:    6 10:14:08    -2273.542930*       2.0395
FIRE:    7 10:14:08    -2273.636330*       1.1842
FIRE:    8 10:14:09    -2273.709910*       1.9144
FIRE:    9 10:14:10    -2273.761850*       2.5581
FIRE:   10 10:14:10    -2273.808580*       1.7936
FIRE:   11 10:14:11    -2273.850060*       1.0683
FIRE:   12 10:14:12    -2273.887840*       1.4766
FIRE:   13 10:14:12    -2273.944840*       2.0809
FIRE:   14 10:14:13    -2274.030750*       1.4074
FIRE:   15 10:14:14    -2274.109620*       1.1815
FIRE:   16 10:14:14    -2274.156400*       1.7897
FIRE:   17 10:14:15    -2274.213350*       0.7744
FI

0

In [34]:
from optimization_methods import mopac_opt

mopac_opt(ts_coords, data.atomnos, method='PM7 FORCE', title='ts', read_output=False)
os.system('avogadro ts.out')
# test the obtained TS (force MOPAC calc)

0

In [71]:
from tscode import suppress_stdout_stderr

for scale_fmax in (1, 1.5, 2, 2.5, 3):
    with timer('scale_fmax = ' + str(scale_fmax)):
        with suppress_stdout_stderr():
            ase_neb(start, end, data.atomnos, scale_fmax=scale_fmax)


   4.3897
LBFGS:   62 11:51:30    -2274.011770*       0.7083
LBFGS:   63 11:51:30    -2274.021370*       0.9730
LBFGS:   64 11:51:30    -2273.926260*       1.7031
LBFGS:   65 11:51:30    -2273.917260*       0.9526
LBFGS:   66 11:51:31    -2273.979600*       0.8647
LBFGS:   67 11:51:31    -2273.992520*       2.2989
LBFGS:   68 11:51:31    -2273.407760*       4.0881
LBFGS:   69 11:51:31    -2273.348580*       2.8675
LBFGS:   70 11:51:32    -2274.108740*       1.2720
LBFGS:   71 11:51:32    -2274.058480*       1.3400
LBFGS:   72 11:51:32    -2273.920470*       2.0851
LBFGS:   73 11:51:33    -2273.948000*       0.4209
LBFGS:   74 11:51:33    -2273.972430*       0.5203
LBFGS:   75 11:51:33    -2273.984450*       0.6455
LBFGS:   76 11:51:33    -2273.806500*       1.0651
LBFGS:   77 11:51:33    -2273.943680*       0.3256
LBFGS:   78 11:51:34    -2273.959720*       0.1303
LBFGS:   79 11:51:34    -2273.958780*       0.2245
LBFGS:   80 11:51:34    -2273.964960*       0.1050
LBFGS:   81 11:51:34 

KeyboardInterrupt: 

## Get reagents and products from pseudo-TS

In [1]:
from optimization_methods import mopac_opt, write_xyz
from optimization_methods import get_reagent, get_product
from cclib.io import ccread
import numpy as np
import os
os.chdir('Resources/tri')
data = ccread('TSCoDe_TSs_guesses.xyz')
ids = np.array([32, 5, 0])
constrained_indexes = np.array([[0, 43], [5, 36], [33, 60]])

In [2]:
%%time
reagent_coords = get_reagent(data.atomcoords[0], data.atomnos, ids, constrained_indexes)
with open('reagents.xyz', 'w') as f:
    write_xyz(reagent_coords, data.atomnos, f)

Wall time: 10.8 s


In [3]:
os.system('avogadro reagents.xyz')

0

In [3]:
%%time
product_coords = get_product(data.atomcoords[0], data.atomnos, ids, constrained_indexes)
with open('products.xyz', 'w') as f:
    write_xyz(product_coords, data.atomnos, f)

Wall time: 34.2 s


In [4]:
os.system('avogadro products.xyz')

0

In [4]:
import time
from ase.optimize import LBFGS
from ase.neb import idpp_interpolate
from ase.dyneb import DyNEB
from ase import Atoms
from optimization_methods import ase_neb

class timer():
    def __init__(self, title=''):
        self.title = title
    def __enter__(self):
        self.t1 = time.time()
    def __exit__(self, type, value, traceback):
        self.t2 = time.time()
        if self.title != '':
            s = f'{self.title} - {round(self.t2-self.t1, 3)} s'
        else:
            s = f'{round(self.t2-self.t1, 3)} s' 
        print(s)

def ase_neb_test(reagents, products, atomnos, n_images=6, fmax=0.05, method='PM7 GEO-OK', title='temp', optimizer=LBFGS):

    # Read initial and final states:
    initial = Atoms(atomnos, positions=reagents)
    final = Atoms(atomnos, positions=products)

    # Make a band consisting of n images:
    images = [initial]
    images += [initial.copy() for i in range(n_images)]
    images += [final]


    neb = DyNEB(images, fmax=fmax, climb=False,  method='eb', scale_fmax=1, allow_shared_calculator=True)

    # Interpolate linearly the positions of the middle images:
    # idpp_interpolate(neb, fmax=0.01, steps=7000)
    neb.interpolate()
    dump(f'{title}_MEP_guess.xyz', images, atomnos)


In [3]:
reagents = ccread('reagents.xyz')
products = ccread('products.xyz')
from optimization_methods import ase_neb

# with timer(title=f'NEB'):
#     ts, _ = ase_neb(reagents.atomcoords[0], data.atomcoords[-1], products.atomcoords[0], reagents.atomnos, title='OOF')

# with timer(title=f'FORCE'):
#     mopac_opt(ts, data.atomnos, method='PM7 FORCE LET', title=f'TS_FREQ', read_output=False)

In [6]:
from hypermolecule_class import pt
2*2*pt[6].covalent_radius

3.04

In [7]:
%%time
ts, _ = ase_neb(reagents.atomcoords[0], products.atomcoords[0], reagents.atomnos, n_images=6, title='TEST')
# 2 minutes with 6 images

Wall time: 4min 5s


In [8]:
%%time
mopac_opt(ts, data.atomnos, method='PM7 FORCE LET', title=f'TEST_FREQ', read_output=False)

Wall time: 39.6 s


In [9]:
os.system('avogadro TEST_FREQ.out')

0

## Trimolecular Reag/Prods

In [1]:
from optimization_methods import mopac_opt, write_xyz
from optimization_methods import get_reagent, get_product
from cclib.io import ccread
import numpy as np
import os
os.chdir('Resources/tri')
data = ccread('TS_test.xyz')
ids = (32,35,31)

constrained_indexes = np.array([[15,82], [14,53],
                                # [50,71]
                                ])

In [2]:
%%time
product_coords = get_product(data.atomcoords[-1], data.atomnos, ids, constrained_indexes)
with open('products.xyz', 'w') as f:
    write_xyz(product_coords, data.atomnos, f)

Reactive dist - 2.157352731937918
Reactive dist - 2.057352731148497
Reactive dist - 1.9573527309571888
Reactive dist - 1.8573527307974216
Wall time: 2min 44s


In [3]:
%%time
reagents_coords = get_reagent(data.atomcoords[-1], data.atomnos, ids, constrained_indexes)
with open('reagents.xyz', 'w') as f:
    write_xyz(reagents_coords, data.atomnos, f)

Wall time: 2min


In [2]:
reagents = ccread('reagents.xyz')
products = ccread('products.xyz')
from optimization_methods import ase_neb

In [3]:
%%time
ts, _ = ase_neb(reagents.atomcoords[0], products.atomcoords[0], reagents.atomnos)

Stopped NEB for OOF:
Calculator "mopac" failed with command "mopac2016 temp.mop" failed in c:\Users\ehrma\Desktop\Coding\TSCoDe\Resources\tri with error code 1


CalculationFailed: Calculator "mopac" failed with command "mopac2016 temp.mop" failed in c:\Users\ehrma\Desktop\Coding\TSCoDe\Resources\tri with error code 1

In [ ]:
mopac_opt(ts, data.atomnos, method='PM7 FORCE LET', title=f'TS_FREQ', read_output=False)

In [4]:
os.system('mopac2016')

0

## Trimolecular TSs directions

In [20]:
import numpy as np
from utils import *
def _get_directions(norms):
    '''
    Returns two or three vectors specifying the direction in which each molecule should be aligned
    in the cyclical TS, pointing towards the center of the polygon.
    '''
    assert len(norms) in (2,3)
    if len(norms) == 2:
        return np.array([[0,1,0],
                            [0,-1,0]])
    else:

        norms = sorted(norms)
        vertexes = np.zeros((3,2))

        vertexes[1] = np.array([norms[0],0])

        a = np.power(norms[0], 2)
        b = np.power(norms[1], 2)
        c = np.power(norms[2], 2)
        x = (a-b+c)/(2*a**0.5)
        y = (c-x**2)**0.5

        vertexes[2] = np.array([x,y])
        # similar to the code from polygonize, to get the active triangle
        # but without the orientation specified in the polygonize function
        
        a = vertexes[1,0] # first point, x
        b = vertexes[2,0] # second point, x
        c = vertexes[2,1] # second point, y

        x = a/2
        y = (b**2 + c**2 - a*b)/(2*c)
        cc = np.array([x,y])
        # 2D coordinates of the triangle circocenter

        v0, v1, v2 = vertexes

        meanpoint1 = np.mean((v0,v1), axis=0)
        meanpoint2 = np.mean((v1,v2), axis=0)
        meanpoint3 = np.mean((v2,v0), axis=0)

        dir1 = cc - meanpoint1
        dir2 = cc - meanpoint2
        dir3 = cc - meanpoint3
        # 2D direction versors connecting center of side with circumcenter.
        # Now we need to understand if we want these or their negative

        if np.any([np.all(d == 0) for d in (dir1, dir2, dir3)]):
        # We have a right triangle. To aviod numerical
        # errors, a small perturbation is made.
        # This should not happen, but just in case...
            norms[0] += 1e-10
            dir1, dir2, dir3 = [t[:-1] for t in _get_directions(norms)]

        angle0_obtuse = (angle(v1-v0, v2-v0) > 90)
        angle1_obtuse = (angle(v0-v1, v2-v1) > 90)
        angle2_obtuse = (angle(v0-v2, v1-v2) > 90)

        dir1 = -dir1 if angle2_obtuse else dir1
        dir2 = -dir2 if angle0_obtuse else dir2
        dir3 = -dir3 if angle1_obtuse else dir3
        # invert the versors sign if circumcenter is
        # one angle is obtuse, because then
        # circumcenter is outside the triangle
        
        dir1 = norm(np.concatenate((dir1, [0])))
        dir2 = norm(np.concatenate((dir2, [0])))
        dir3 = norm(np.concatenate((dir3, [0])))

        return np.vstack((dir1, dir2, dir3))

_get_directions((1.001,1.001,1.001))

array([[ 0.       ,  1.       ,  0.       ],
       [-0.8660254, -0.5      ,  0.       ],
       [ 0.8660254, -0.5      ,  0.       ]])

In [21]:
from utils import polygonize
norms = (3,5,4)
polygonize(norms)[0]

array([[[0., 0., 0.],
        [3., 0., 0.]],

       [[3., 0., 0.],
        [0., 4., 0.]],

       [[0., 4., 0.],
        [0., 0., 0.]]])

In [25]:
_get_directions(norms)

array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-1.00000000e+00, -2.50000021e-11,  0.00000000e+00],
       [ 8.00000000e-01, -6.00000000e-01,  0.00000000e+00]])

In [13]:
from utils import cartesian_product
steps = 6
angle_range = 30
angles = cartesian_product(*[range(steps+1) for _ in range(3)]) * 2*angle_range/steps - angle_range
len(angles)
angles

array([[-30., -30., -30.],
       [-30., -30., -20.],
       [-30., -30., -10.],
       ...,
       [ 30.,  30.,  10.],
       [ 30.,  30.,  20.],
       [ 30.,  30.,  30.]])

In [3]:
from utils import polygonize
a = 5
b = 4
c = (a+b)*0.8

polygonize((a,b,c))[0]

array([[[0.        , 0.        , 0.        ],
        [5.        , 0.        , 0.        ]],

       [[5.        , 0.        , 0.        ],
        [6.084     , 3.85031739, 0.        ]],

       [[6.084     , 3.85031739, 0.        ],
        [0.        , 0.        , 0.        ]]])

In [ ]:
def mopac_bend(mol, pivot, threshold, method='PM7', title='temp_scan'):
    '''
    This function writes a MOPAC .mop input, runs it with the subprocess
    module and reads its output. Coordinates used are mixed
    (cartesian and internal) to be able to constrain/sacn the reactive atoms
    distances specified in constrained_indexes.

    :params mol: Hypermolecule Object
    :params method: string, specifiyng the first line of keywords for the MOPAC input file.
    :params title: string, used as a file name and job title for the mopac input file.
    '''
    # TODO:conformations!
    conf = 0

    stepsize = 0.05
    steps = (np.linalg.norm(pivot.pivot)-threshold)//stepsize

    order = []
    s = [method + f' STEP={stepsize} POINT={steps}\n' + title + '\n\n']
    for i, num in enumerate(atomnos):
        s.append(' {} {} +1 {} +1 {} +1\n'.format(pt[num].symbol, *mol.atomcoords[i]))

    id_start, id_end = pivot.index

    r_atom_start = list(mol.reactive_atoms_classes_dict.values())[0]
    orb_start = r_atom_start.center[id_start]

    r_atom_end = list(mol.reactive_atoms_classes_dict.values())[1]
    orb_end = r_atom_end.center[id_end]

    coords = mol.atomcoords[conf]

    ###################################################################

    b1, c1, d1 = np.random.choice(len(coords), 3)
    while b1 == c1 or c1 == d1:
        b1, c1, d1 = np.random.choice(len(coords), 3)
    # indexes of reference atoms

    dist1 = np.linalg.norm(orb_start - coords[b1]) # in Angstrom

    angle1 = np.arccos(norm(orb_start - coords[b1]) @ norm(coords[c1] - coords[b1]))*180/np.pi # in degrees

    d_angle1 = dihedral([orb_start,
                         coords[b1],
                         coords[c1],
                         coords[d1]])
    d_angle1 += 360 if d_angle < 0 else 0


    s.append(' * {}  0 {} +1 {} +1 {} {} {}\n'.format(dist1, angle1, d_angle1 ,b1, c1, d1))

    ###################################################################

    b2, c2, d2 = np.random.choice(len(coords), 3)
    while b2 == c2 or c2 == d2:
        b2, c2, d2 = np.random.choice(len(coords), 3)
    # indexes of reference atoms

    dist2 = np.linalg.norm(orb_start - coords[b2]) # in Angstrom

    angle2 = np.arccos(norm(orb_start - coords[b2]) @ norm(coords[c2] - coords[b2]))*180/np.pi # in degrees

    d_angle2 = dihedral([orb_start,
                         coords[b2],
                         coords[c2],
                         coords[d2]])
    d_angle2 += 360 if d_angle < 0 else 0


    s.append(' * {}  0 {} +1 {} +1 {} {} {}\n'.format(dist2, angle2, d_angle2 ,b2, c2, d2))
    
    ###################################################################
    
    c3, d3 = np.random.choice(len(coords), 2)
    while c3 == d3:
        c3, d3 = np.random.choice(len(coords), 2)
    # indexes of reference atoms

    dist3 = np.linalg.norm(orb_start - orb_end) # in Angstrom

    angle3 = np.arccos(norm(orb_start - orb_end) @ norm(orb_end - coords[c3]))*180/np.pi # in degrees

    d_angle2 = dihedral([orb_start,
                         orb_end,
                         coords[c3],
                         coords[d3]])
    d_angle2 += 360 if d_angle < 0 else 0

    len_s = len(s)
    s.append(' * {} -1 {} +1 {} +1 {} {} {}\n'.format(dist3, angle3, d_angle3 , len_s, c3, d3))
    
    ###################################################################

    s = ''.join(s)
    with open(f'{title}.mop', 'w') as f:
        f.write(s)
    
    # try:
    #     check_call(f'{MOPAC_COMMAND} {title}.mop'.split(), stdout=DEVNULL, stderr=STDOUT)
    # except KeyboardInterrupt:
    #     print('KeyboardInterrupt requested by user. Quitting.')
    #     quit()

    # os.remove(f'{title}.mop')
    # # delete input, we do not need it anymore

    # if read_output:

    #     inv_order = [order.index(i) for i in range(len(order))]
    #     # undoing the atomic scramble that was needed by the mopac input requirements

    #     opt_coords, energy, success = read_mop_out(f'{title}.out')
    #     os.remove(f'{title}.out')

    #     opt_coords = scramble(opt_coords, inv_order) if opt_coords is not None else coords
    #     # If opt_coords is None, that is if TS seeking crashed,
    #     # sets opt_coords to the old coords. If not, unscrambles
    #     # coordinates read from mopac output.

    #     return opt_coords, energy, success


In [3]:
import numpy as np
from ase import Atoms
from ase.visualize import view
from optimization_methods import Spring
from ase.optimize.lbfgs import LBFGS
from ase.calculators.mopac import MOPAC

pos = np.array([[0,0,1], [0,0,-1]])
atoms = Atoms('HH', positions=pos)
atoms.set_constraint(Spring(0,1,3,100))
atoms.calc = MOPAC(label='temp_H2', command='MOPAC2016 temp_H2.mop')
opt = LBFGS(atoms, maxstep=0.1)
opt.run(fmax=0.05, steps=100)
view(atoms)

       Step     Time          Energy         fmax
LBFGS:    0 09:22:12      -20.133050       95.7932
LBFGS:    1 09:22:12      -19.383150       76.6775
LBFGS:    2 09:22:12      -18.792460       57.3885
LBFGS:    3 09:22:12      -18.328450       37.9486
LBFGS:    4 09:22:12      -17.963430       18.3826
LBFGS:    5 09:22:12      -17.690280        0.0931
LBFGS:    6 09:22:12      -17.691520        0.0002


## NCI print

In [1]:
from optimization_methods import get_nci
from cclib.io import ccread
import numpy as np
import os
os.chdir('Resources/tri')
data = ccread('TSCoDe_TSs_guesses.xyz')
ids = np.array([32, 5, 31])
constrained_indexes = np.array([[0, 43], [5, 36], [33, 60]])

In [3]:
get_nci(data.atomcoords[0], data.atomnos, constrained_indexes, ids)[0]

[]

In [ ]:
os.chdir('../knoevenagel')
data = ccread('Knoevenagel-major.xyz')
ids = np.array([32, 5, 31])
constrained_indexes = np.array([[0, 43], [5, 36], [33, 60]])

## Scramble test

In [1]:
from optimization_methods import scramble_check
from hypermolecule_class import graphize
from cclib.io import ccread
import numpy as np
import os
os.chdir('Resources/epox')
data = ccread('TSCoDe_TSs_guesses.xyz')
alkene = ccread('C2H4.xyz')
peracid = ccread('HCOOOH.xyz')
graphs = [graphize(mol.atomcoords[0], mol.atomnos) for mol in (alkene, peracid)]

In [3]:
results = []
for s in data.atomcoords:
    results. append(scramble_check(s, data.atomnos, graphs, max_newbonds=1))
results

[False, True]

## REAG/PROD CHELOTROPIC

In [1]:
from optimization_methods import mopac_opt, write_xyz
from optimization_methods import get_reagent, get_product
from cclib.io import ccread
import numpy as np
import os
os.chdir('Resources/epox')
data = ccread('TSCoDe_TSs_guesses.xyz')
ids = (6,6)

constrained_indexes = np.array([[0, 10], [1, 10]])

In [2]:
%%time
product_coords = get_product(data.atomcoords[0], data.atomnos, ids, constrained_indexes)
with open('products.xyz', 'w') as f:
    write_xyz(product_coords, data.atomnos, f)

Wall time: 13.4 s


In [3]:
os.system('avogadro products.xyz')

0

In [4]:
%%time
reagents_coords = get_reagent(data.atomcoords[0], data.atomnos, ids, constrained_indexes)
with open('reagents.xyz', 'w') as f:
    write_xyz(reagents_coords, data.atomnos, f)

Wall time: 980 ms


In [5]:
os.system('avogadro reagents.xyz')

0

In [ ]:
reagents = ccread('reagents.xyz')
products = ccread('products.xyz')
from optimization_methods import ase_neb

In [ ]:
%%time
ts, _ = ase_neb(reagents.atomcoords[0], products.atomcoords[0], reagents.atomnos)

In [ ]:
mopac_opt(ts, data.atomnos, method='PM7 FORCE LET', title=f'TS_FREQ', read_output=False)

## checks

In [1]:
from prune import prune_conformers
from cclib.io import ccread
import os
os.chdir('Resources/epox')
data = ccread('TSCoDe_TSs_guesses_May_24_19-58.xyz')

In [14]:
prune_conformers(data.atomcoords, data.atomnos, max_rmsd=1, k=1)[1]

array([ True,  True, False, False])

In [17]:
from spyrmsd.rmsd import rmsd
rmsd(data.atomcoords[0], data.atomcoords[1], data.atomnos, data.atomnos, center=True, minimize=True)

1.442452180419543

## Alignment test

In [1]:
from cclib.io import ccread
from hypermolecule_class import align_structures
from optimization_methods import write_xyz
import numpy as np
import os
os.chdir('Resources/tri')

data = ccread('TSCoDe_TSs_guesses_May_24_23-36.xyz')

aligned = align_structures(np.array(data.atomcoords), indexes=np.array([[0, 43], [5, 36], [33, 60]]))

with open('aligned_i_hope.xyz', 'w') as f:
    for s in aligned:
        write_xyz(s, data.atomnos, f)

os.system('avogadro aligned_i_hope.xyz')

0

In [1]:
from cclib.io import ccread
from hypermolecule_class import align_structures
from optimization_methods import write_xyz
import numpy as np
import os
os.chdir('Resources/tri')

ccread('temp_ob_out.xyz').atomcoords.shape

(1, 68, 3)